In [1]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import scipy.optimize

In [2]:
import astro_time
import sgp4
import sensor

import tle_fitter

In [3]:
import public_astrostandards as PA
PA.init_all()
# PA.get_versions()

0

In [4]:
# grab some raw UDL obs; convert the time; sort
def prepObs( o_df ):
    o_df['obTime_dt'] = pd.to_datetime( o_df['obTime'] )
    o_df.sort_values(by='obTime_dt', inplace=True )
    t_df = astro_time.convert_times( obs['obTime_dt'], PA )
    o_df = pd.concat( (t_df.reset_index(drop=True),o_df.reset_index(drop=True) ), axis=1 )
    return o_df 
    
obs    = pd.read_json('./19548.json.gz').sort_values(by='obTime').reset_index(drop=True)
obs_df = prepObs( obs )    

In [5]:
def rotateTEMEObs( O , harness ):
    '''
    given an ob (O) with ra / declination fields (J2K), convert to TEME
    '''
    newRA  = (harness.ctypes.c_double)()
    newDec = (harness.ctypes.c_double)()
    PA.AstroFuncDll.RotRADec_EqnxToDate( 106,
                                        2,
                                        O['ds50_utc'],
                                        O['ra'],
                                        O['declination'],
                                        newRA,
                                        newDec )
    return ( np.float64(newRA),np.float64(newDec) )

# rotate a dataframe of obs into TEME and then also get a TEME look vector (for solving)
def rotateTEMEdf( df, harness ):
    tv = df.apply( lambda X : rotateTEMEObs( X, harness ) , axis=1 )
    df['teme_ra']  = [ X[0] for X in tv ]
    df['teme_dec'] = [ X[1] for X in tv ]
    x = np.cos( np.radians(df['teme_dec']) ) * np.cos( np.radians( df['teme_ra'] ) )
    y = np.cos( np.radians(df['teme_dec']) ) * np.sin( np.radians( df['teme_ra'] ) )
    z = np.sin( np.radians(df['teme_dec'] ) )
    lv = np.hstack( ( x.values[:,np.newaxis], y.values[:,np.newaxis], z.values[:,np.newaxis] )  )
    df['teme_lv'] = lv.tolist()
    return df

obs_df = rotateTEMEdf( obs_df, PA )[['ra','teme_ra']]

In [6]:
L1 = '1 19548U 88091B   25281.05493527 -.00000297  00000+0  00000+0 0  9993'
L2 = '2 19548  12.7961 342.6596 0038175 340.8908  24.1736  1.00278194122860'

In [7]:
# -----------------------------------------------------------------------------------------------------
def optFunction( X, EH, return_scalar=True ):
    PA      = EH.PA
    XS_TLE  = PA.Cstr('',512)
    # take the function parameters (X) and overwrite the "new_tle" values based on FIELDS 
    for k,v in zip(EH.FIELDS,X) :  EH.new_tle[ k ] = v
    # --------------------- clear state
    PA.TleDll.TleRemoveAllSats()
    PA.Sgp4PropDll.Sgp4RemoveAllSats()
    # --------------------- init our test TLE from the modified data
    tleid = PA.TleDll.TleAddSatFrArray( EH.new_tle.data, XS_TLE )
    if tleid <= 0: return np.inf
    if PA.Sgp4PropDll.Sgp4InitSat( tleid ) != 0: return np.inf
    # --------------------- generate our test ephemeris
    test_eph = sgp4.propTLEToDS50s( tleid, EH._ds50_utc, PA )
    # these are our test look vectors
    test_lv  = test_eph[:,1:4] - EH._sensor_eci
    # normalize 
    test_lv  = test_lv / np.linalg.norm( test_lv, axis=1)[:,np.newaxis]
    rv =  np.sum( np.abs( np.sum( test_lv * EH._look_vecs, axis=1 ) ) )
    print('{:8.3f}                 '.format(rv), end='\r')
    return rv

In [8]:
# -----------------------------------------------------------------------------------------------------
class eo_fitter( tle_fitter.tle_fitter ):
    def __init__( self, PA ):
        super().__init__( PA )
        self.line1 = None
        self.line2 = None

    def _get_sv_at_epoch( self ):
        ''' assume that epoch is set, and that line1, line2 are also set '''
        self.PA.TleDll.TleRemoveAllSats()
        tleid   = sgp4.addTLE( self.line1, self.line2, self.PA )
        assert sgp4.initTLE( tleid, self.PA )
        rv      = sgp4.propTLEToDS50s( tleid, [ self.epoch_ds50 ], self.PA )[0]
        return  { 'teme_p' : rv[1:4], 'teme_v' : rv[4:7], 'ds50_utc' : self.epoch_ds50 }

    def _init_obs( self ):
        self.obs_df     = prepObs( obs )
        self.obs_df     = rotateTEMEdf( self.obs_df, self.PA )
        self._look_vecs = np.vstack( self.obs_df['teme_lv'] )
        self._ds50_utc  = self.obs_df['ds50_utc'].values
    
    def _init_tle( self, epoch_idx=-1 ):
        # init the TLE from the lines data
        self.init_tle    = tle_fitter.TLE_str_to_XA_TLE( L1, L2, self.PA )
        self.new_tle     = tle_fitter.TLE_str_to_XA_TLE( L1, L2, self.PA )
        print( '\n'.join(tle_fitter.XA_TLE_to_str( self.new_tle, self.PA ) ))
        self.epoch_ds50  = self.obs_df.iloc[ epoch_idx ]['ds50_utc']
        epoch_sv         = self._get_sv_at_epoch()
        osc_data         = tle_fitter.sv_to_osc( epoch_sv , self.PA )
        mean_data        = tle_fitter.osc_to_mean( osc_data, self.PA )
        # update our "fit" TLE with the new osculating data
        self.new_tle    = tle_fitter.insert_kep_to_TLE( self.new_tle, mean_data, self.PA )
        self.new_tle['XA_TLE_EPOCH'] = epoch_sv['ds50_utc']
        # if this is a type-0, we need Kozai mean motion   
        if self.new_tle['XA_TLE_EPHTYPE'] == 0:
            self.new_tle['XA_TLE_MNMOTN'] = self.PA.AstroFuncDll.BrouwerToKozai( 
                                                self.new_tle['XA_TLE_ECCEN'], 
                                                self.new_tle['XA_TLE_INCLI'],
                                                self.new_tle['XA_TLE_MNMOTN'] )

        print( '\n'.join(tle_fitter.XA_TLE_to_str( self.new_tle, self.PA ) ))
        return self

    def _init_sensor( self ):
        self.sensor_df        = self.obs_df[['ds50_utc','senlat','senlon','senalt']].rename( columns = {'senlat' : 'lat','senlon' : 'lon', 'senalt' : 'height' } )
        self.sensor_df        = sensor.llh_to_eci( self.sensor_df, self.PA )
        self._sensor_eci      = np.vstack( self.sensor_df['teme_p'].values )
    
    def _init_fields( self ):
        # DO NOT change the order.. they build
        self._init_obs()
        self._init_sensor()
        self._init_tle()

    def set_data( self, L1 : str, L2 : str, obs : list[ dict ] ):
        ''' 
        take an initial TLE as a guess (L1,L2) 
        take a list of JSON formatted obs (directly from UDL)
        solve for a new TLE
        '''
        self.line1      = L1
        self.line2      = L2
        self.obs        = obs
        self._init_fields()
        return self

    def initial_simplex( self, delta=0.2):
        '''
        take our initial fields and perturb each entry delta% in either direction (up and down
        this should give us a good search space
        '''
        X     = self.get_init_fields()
        smplx = np.ones( shape=( len(X), len(X) ) )
        smplx += np.diag( np.ones( len(X)-1 ), -1 ) * -delta
        smplx += np.diag( np.ones( len(X)-1 ), 1 ) * delta
        smplx = np.vstack( (np.ones(len(X)), smplx ) )
        smplx *= X
        return smplx
                                            

obs    = pd.read_json('./19548.json.gz').sort_values(by='obTime').reset_index(drop=True)    
A = eo_fitter( PA ).set_data( L1, L2, obs )

optFunction( A.get_init_fields(), A )


# # -----------------------------  nelder mead -----------------------------
# # if your seed is not near the final, nelder works great (at the expense of time)
# ans   = scipy.optimize.minimize(optFunction, 
#                                 A.get_init_fields(),
#                                 args    = (A,True),
#                                 method  = 'Nelder-Mead' ,
#                                 #options = {'xatol' : 0.01, 'fatol' : 0.9 } )
#                                 options = {'xatol' : 0.01, 'fatol' : 0.1, 'initial_simplex' : A.initial_simplex() } )
#                                #options = {'initial_simplex' : smplx } )


1 19548           25281.05493527 -.00000297  00000 0  00000 0 0 0999
2 19548  12.7961 342.6596 0038175 340.8908  24.1736  1.0027819412286
1 19548           25282.37703627 -.00000297  00000 0  00000 0 0 0999
2 19548  12.7846 342.6148 0038111 341.2730 141.1035  1.0027767012286
 817.399                 

np.float64(817.3993847485506)

In [9]:
ans

NameError: name 'ans' is not defined

In [ ]:
A.get_init_fields()